In [36]:
import pandas as pd
import numpy as np

In [37]:
!wget -O data_collisions.csv https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv

--2020-09-25 02:33:52--  https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv
Resolving s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)... 67.228.254.196
Connecting to s3.us.cloud-object-storage.appdomain.cloud (s3.us.cloud-object-storage.appdomain.cloud)|67.228.254.196|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 73917638 (70M) [text/csv]
Saving to: ‘data_collisions.csv’

data_collisions.csv 100%[===================>]  70.49M  11.6MB/s    in 5.5s    

2020-09-25 02:33:57 (12.7 MB/s) - ‘data_collisions.csv’ saved [73917638/73917638]



In [38]:
df = pd.read_csv('data_collisions.csv')
df.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [39]:
df1 = df[['SEVERITYCODE','WEATHER','ROADCOND','LIGHTCOND']]
df1

,SEVERITYCODE,WEATHER,ROADCOND,LIGHTCOND
0,2,Overcast,Wet,Daylight
1,1,Raining,Wet,Dark - Street Lights On
2,1,Overcast,Dry,Daylight
3,1,Clear,Dry,Daylight
4,2,Raining,Wet,Daylight
...,...,...,...,...
194668,2,Clear,Dry,Daylight
194669,1,Raining,Wet,Daylight
194670,2,Clear,Dry,Daylight
194671,2,Clear,Dry,Dusk


In [40]:
#Required for pre-processing to determine imbalance in data

df1["SEVERITYCODE"].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [41]:
from sklearn.utils import resample

#Required to resolve imbalance of data

df_high = df1[df.SEVERITYCODE==1]
df_low = df1[df.SEVERITYCODE==2]

df_high_dsample = resample(df_high, 
                           replace=False,
                           n_samples=58188,
                           random_state=123)

balanced_df = pd.concat([df_high_dsample, df_low])

balanced_df.SEVERITYCODE.value_counts()

2    58188
1    58188
Name: SEVERITYCODE, dtype: int64

In [42]:
df1["WEATHER"].value_counts()

Clear                       111135
Raining                      33145
Overcast                     27714
Unknown                      15091
Snowing                        907
Other                          832
Fog/Smog/Smoke                 569
Sleet/Hail/Freezing Rain       113
Blowing Sand/Dirt               56
Severe Crosswind                25
Partly Cloudy                    5
Name: WEATHER, dtype: int64

In [43]:
df1["LIGHTCOND"].value_counts()

Daylight                    116137
Dark - Street Lights On      48507
Unknown                      13473
Dusk                          5902
Dawn                          2502
Dark - No Street Lights       1537
Dark - Street Lights Off      1199
Other                          235
Dark - Unknown Lighting         11
Name: LIGHTCOND, dtype: int64

In [44]:
df1["ROADCOND"].value_counts()

Dry               124510
Wet                47474
Unknown            15078
Ice                 1209
Snow/Slush          1004
Other                132
Standing Water       115
Sand/Mud/Dirt         75
Oil                   64
Name: ROADCOND, dtype: int64

In [45]:
from sklearn import preprocessing
 
label_encoder = preprocessing.LabelEncoder()

# Encode labels in columns WEATHER, ROADCOND, and LIGHTCOND. 
df1['WEATHER_CATEGORY']= label_encoder.fit_transform(df1['WEATHER'].astype(str)) 
df1['ROADCOND_CATEGORY']= label_encoder.fit_transform(df1['ROADCOND'].astype(str)) 
df1['LIGHTCOND_CATEGORY']= label_encoder.fit_transform(df1['LIGHTCOND'].astype(str)) 
print(df1.head())

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


   SEVERITYCODE   WEATHER ROADCOND                LIGHTCOND  WEATHER_CATEGORY  \
0             2  Overcast      Wet                 Daylight                 4   
1             1   Raining      Wet  Dark - Street Lights On                 6   
2             1  Overcast      Dry                 Daylight                 4   
3             1     Clear      Dry                 Daylight                 1   
4             2   Raining      Wet                 Daylight                 6   

   ROADCOND_CATEGORY  LIGHTCOND_CATEGORY  
0                  8                   5  
1                  8                   2  
2                  0                   5  
3                  0                   5  
4                  8                   5  


/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
df1.dtypes

SEVERITYCODE           int64
WEATHER               object
ROADCOND              object
LIGHTCOND             object
WEATHER_CATEGORY       int64
ROADCOND_CATEGORY      int64
LIGHTCOND_CATEGORY     int64
dtype: object

In [47]:
# Remove object columns
df2 = df1[['SEVERITYCODE','WEATHER_CATEGORY','ROADCOND_CATEGORY','LIGHTCOND_CATEGORY']]
df2

,SEVERITYCODE,WEATHER_CATEGORY,ROADCOND_CATEGORY,LIGHTCOND_CATEGORY
0,2,4,8,5
1,1,6,8,2
2,1,4,0,5
3,1,1,0,5
4,2,6,8,5
...,...,...,...,...
194668,2,1,0,5
194669,1,6,8,5
194670,2,1,0,5
194671,2,1,0,6


In [48]:
# Identify X

X = df2
X[0:5]

,SEVERITYCODE,WEATHER_CATEGORY,ROADCOND_CATEGORY,LIGHTCOND_CATEGORY
0,2,4,8,5
1,1,6,8,2
2,1,4,0,5
3,1,1,0,5
4,2,6,8,5


In [49]:
# Identify y

y = df2['SEVERITYCODE'].values
y[0:5]

array([2, 1, 1, 1, 2])

In [50]:
# Setting up for testing
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=7 )

print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)

Train set: (136271, 4) (136271,)
Test set: (58402, 4) (58402,)


In [51]:
# KNN Model

from sklearn.neighbors import KNeighborsClassifier

k = 16

knn = KNeighborsClassifier(n_neighbors = k).fit(X_train,y_train)

knn_target = knn.predict(X_test)
knn_target[0:5]

array([1, 1, 1, 1, 1])

In [52]:
from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import f1_score
from sklearn.metrics import log_loss

# Jaccard score
jaccard_knn = jaccard_similarity_score(y_test, knn_target)
print("KNN Jaccard index: ", jaccard_knn)

# F1 score
f1_score_knn = f1_score(y_test, knn_target, average='weighted')
print("KNN F1-score: ", f1_score_knn)

KNN Jaccard index:  0.9986644293003665
KNN F1-score:  0.9986635524919164


In [53]:
#Decision Tree

from sklearn.tree import DecisionTreeClassifier
model_tree = DecisionTreeClassifier(criterion="entropy", max_depth = 7)
model_tree.fit(X_train,y_train)

model_tree

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=7,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [54]:
yhat_dt = knn.predict(X_test)

# Jaccard score
jaccard_dt = jaccard_similarity_score(y_test, yhat_dt)
print("DT Jacard index: ", jaccard_dt)

# f1_score
f1_score_dt = f1_score(y_test, yhat_dt, average='weighted')
print("DT F1-score: ", f1_score_dt)

DT Jacard index:  0.9986644293003665
DT F1-score:  0.9986635524919164


In [55]:
# Logistic Regression

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
model_lr = LogisticRegression(C=6, solver='liblinear').fit(X_train,y_train)
model_lr


LogisticRegression(C=6, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='liblinear',
          tol=0.0001, verbose=0, warm_start=False)

In [56]:
yhat_lg = knn.predict(X_test)
yhat_lg_prob = knn.predict_proba(X_test)

# Jaccard score
jaccard_lg = jaccard_similarity_score(y_test, yhat_lg)
print("LR Jaccard Index: ", jaccard_lg)

# f1_score
f1_score_lg = f1_score(y_test, yhat_lg, average = 'weighted')
print("LR F1-score: ", f1_score_lg)

# Log Loss
logloss_lg = log_loss(y_test, yhat_lg_prob)
print("LR log loss: ", logloss_lg)


LR Jaccard Index:  0.9986644293003665
LR F1-score:  0.9986635524919164
LR log loss:  0.002629290065955424


In [58]:
report = pd.DataFrame(data=np.array([["KNN", jaccard_knn, f1_score_knn, 'NA'],
                                     ["Decision Tree", jaccard_dt, f1_score_dt, 'NA'],
                                    ["LogisticRegression", jaccard_lg, f1_score_lg, logloss_lg]]), columns=["Algorithm", "Jaccard", "F1-score", "LogLoss"])
report = report.set_index(["Algorithm", "Jaccard", "F1-score", "LogLoss"])

report

,,,
Algorithm,Jaccard,F1-score,LogLoss
KNN,0.564001947698565,0.5401775308974308,NA
Decision Tree,0.5664365709048206,0.5450597937389444,NA
LogisticRegression,0.5260218256809784,0.511602093963383,0684953583198887
